Домашнее задание по АОТ

In [1]:
! pip install requests
! pip install nltk
! pip install pymorphy2

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 8.0MB 11.3MB/s 


In [47]:
import requests
import re
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Я взяла сайт LiveLib, отзывы на книгу "девушка в поезде".

Напишем штуку, которая их собирает

In [6]:
def html_collect():
  result = requests.get('https://www.livelib.ru/book/1001526145/reviews-devushka-v-poezde-pola-hokins')
  result.encoding = 'utf-8'
  prob = result.text

  html = [prob]
  for i in range(2, 20):
    link = 'https://www.livelib.ru/book/1001526145/reviews/~'+str(i)+'#reviews'
    result = requests.get(link)
    result.encoding = 'utf-8'
    html.append(result.text)
  return(html)

Один раз соберем их чтобы лишний раз потом не дергать

In [7]:
temp = html_collect()

Эта функция ищет в коде страницы то, что нам нужно: сами отзывы и оценки

In [8]:
def rev_search(html):
  revs = []
  for i in html:
    rev_all = re.compile('<span class="lenta-card__mymark">(.*?)</span>.*?<div id="lenta-card__text-review-escaped">.*?<p><p>(.*?)</p></p>', flags = re.DOTALL)
    revs.append(rev_all.findall(i))
  return(revs)

Эта функция аккуратно собирает их в список

In [32]:
def make_rev_list(revs):
  revlist = []
  for i in revs[0]:
    revlist.append(i)
  for i in range(1, len(revs)):
    for j in range(1, len(revs[i])):
      revlist.append(revs[i][j])
  return(revlist)

Эта функция чистит содержимое от тегов и переносов строки

In [9]:
def rev_clean(revs):
  clean_revs=[]
  for i in revs:
    x = re.sub(r'\n', '', i[0])
    x = re.sub(' ', '', x)
    y = re.sub('<.*?>', '', i[1])
    y = re.sub('\n', ' ', y)
    y = re.sub('  ', ' ', y)
    clean_revs.append([y, x])
  return(clean_revs)

Эта функция считает, что оценка 4 или выше это положительный отзыв, 2.5 или ниже --- отрицательный

In [27]:
def rating(x):
  if float(x) >= 4:
    r = 'good'
  elif float(x) <= 2.5:
    r = 'bad'
  else:
    r = ''
  return(r)

Эта функция находит создает списки положительных и отрицательных рецензий

In [28]:
def find_polar(revs):
  good = [i for i in revs if rating(i[1]) == 'good']
  bad = [i for i in revs if rating(i[1]) == 'bad']
  return(good, bad)

Эта функция делает нам списки из 30 положительных и 30 отрицательных отзывов как надо в задании

In [42]:
def make_data(good, bad):
  good1 = good[:31]
  del good1[24]
  bad1 = bad[:30]
  test1 = good[35:40] + bad[31:36]
  return good1, bad1, test1

Эта функция разбивает отзывы определенной полярности на слова, приводит их к нижнему регистру, чистит и лемматизирует

In [93]:
def polar_words(polar):
  pwords = []
  for i in polar:
    pw = word_tokenize(i[0])
    pwords.extend(pw)
  smallpwords = [w.lower() for w in pwords if w.isalpha()]
  polarwords = [morph.parse(w)[0].normal_form for w in smallpwords] 
  return polarwords

Эта функция чистит списки от слов, которые есть в списке другой полярности (то есть оставляет только слова из положительных или отрицательных отзывов)

In [58]:
def only_words(polar1, polar2):
  return [w for w in polar1 if w not in polar2]

Эта функция убирает слова, которые встречались меньше 3 раз

In [120]:
def only_freq(polar):
  pcounter = Counter(polar)
  return [w for w, val in pcounter.items() if val > 2]

Эта функция считает полярность отзыва. Она считает сколько в нем положительных слов к общему числу положительных слов и аналогично для отрицательных. Так логичнее, потому что отрицательных слов получилось больше, чем положительных.

In [98]:
def polar(review, good, bad):
  rg = 0
  rb = 0
  wrd = word_tokenize(review)
  wrd1 = [w.lower() for w in wrd if w.isalpha()]
  lms = [morph.parse(w)[0].normal_form for w in wrd1]
  for i in lms:
    if i in good:
      rg += 1
    elif i in bad:
      rb += 1
  if rg/len(good)>rb/len(bad):
    p = 'good'
  else:
    p = 'bad'
  return(p)

Эта функция считает точность для тестового набора отзывов.

In [105]:
def accuracy(test, good, bad):
  acc = 0
  for i in test:
    if polar(i[0], good, bad) == rating(i[1]):
      acc += 1
  return(acc/len(test))

Здесь мы применяем все функции по очереди чтоб получить результат.

In [121]:
reviews = rev_clean(make_rev_list(rev_search(temp))) # все отзывы
good0, bad0 = find_polar(reviews) # разделили на хорошие и плохие
goodrevs, badrevs, test = make_data(good0, bad0) # создали данные
goodwords = polar_words(goodrevs) # нашли слова из хороших отзывов
badwords = polar_words(badrevs) # нашли слова из плохих отзывов
freqgood = only_freq(only_words(goodwords, badwords)) # взяли только частотные и только из хороших
freqbad = only_freq(only_words(badwords, goodwords)) # взяли только частотные и только из плохих
accuracy(test, freqgood, freqbad) # посчитали точность

0.7

Ура, неплохая точность, 7/10! Но вряд ли точности, подсчитанной на 10 отзывах можно доверять :)

Как улучшить программу?

Во-первых, как всегда, взять гораааздо больше рецензий. После этого игра с весами (насколько нечастотные слова можно оставить) имела бы больше смысла (я играла, в моем случае >2 оптимальный вариант).

Во-вторых можно попробовать находить частотные для только хороших или плохих отзывов биграммы. Не факт что это поможет, например, в плохих отзывах будет биграмма "ужасная книга", но слово "ужасный" скорее всего и так относится к плохим словам. 